In [1]:
import gym
import gym_anytrading
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
    
plt.ion()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [3]:
class DQN(nn.Module):

    def __init__(self, input_size,outputs):
        super(DQN, self).__init__()
        self.lin1 = nn.Linear(input_size, input_size*2)
        self.lin2 = nn.Linear(input_size*2, input_size)
        self.lin3 = nn.Linear(input_size, outputs)

    def forward(self, x):
        x = F.relu(self.lin1(x.float()))
        x = F.relu(self.lin2(x.float()))
        x = F.softmax(self.lin3(x.float()))
        return x

In [4]:
BATCH_SIZE = 16
GAMMA = 0.999
EPS_START = 0.99
EPS_END = 0.04
EPS_DECAY = 200
TARGET_UPDATE = 20

env = gym.make('forex-v1')

# Get number of actions from gym action space
n_actions = env.action_space.n

observation_size = env.observation_space.shape[0] * (env.observation_space.shape[1]-1)

policy_net = DQN(observation_size, n_actions).to(device)
target_net = DQN(observation_size, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(50000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

c:\users\dark0ne\documents\gym\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [5]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    next_states = [tens.view(-1,batch.next_state[0].size()[0]*batch.next_state[0].size()[1]) for tens in batch.next_state]
    non_final_next_states = torch.cat([s for s in next_states
                                                if s is not None])
    state_batch = torch.cat([tens.view(-1,batch.state[0].size()[0]*batch.state[0].size()[1]) for tens in batch.state])
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [6]:
num_episodes = 1
episode_durations = []

for i_episode in range(num_episodes):
    # Initialize the environment and state
    env.reset()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    state = torch.tensor(state.drop('Timestamp',axis=1).values).view(1,-1)
    for t in count():
        # Select and perform an action
        action = select_action(state)
        next_state, reward, done, info = env.step(action.item())
        next_state = torch.tensor(next_state.drop('Timestamp',axis=1).values).view(1,-1)
        reward = torch.tensor([reward],device=device)

        
        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            break
    # Update the target network, copying all weights and biases in DQN
    print("Episode {} done.".format(i_episode))
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')

c:\users\dark0ne\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Episode 0 done.
Complete


In [8]:
env.max_possible_profit()

2.483316171811293

In [9]:
info

{'total_reward': -162.19999999999462,
 'total_profit': 0.956617144122035,
 'position': 1}